In [1]:
import pandas as pd
# Training Data
training_df = pd.read_csv(r"C:\Users\ebrady\Personal Projects\tweet-sentiment\data\twitter_training.csv")

# Validation Data
validation_df = pd.read_csv(r"C:\Users\ebrady\Personal Projects\tweet-sentiment\data\twitter_validation.csv")

In [2]:
# Visualize the training data
training_df.sample(frac=1).head(10)

,ID,SOURCE,SENTIMENT,TWEET
33355,6526,Fortnite,Negative,bro fuck the bots in Fortnite This game is so ...
64505,7850,MaddenNFL,Negative,why is madden nfl's music this year up so smack
26513,954,AssassinsCreed,Positive,in
39258,5539,Hearthstone,Negative,Holy shit. Arena sucks now. Bring back all the...
5496,147,Amazon,Neutral,Why Instacart and Amazon workers are going on ...
68335,3702,Cyberpunk2077,Neutral,Me.... but..... why.... okay.... so Cyberpunk ...
1328,2632,Borderlands,Positive,. L I V E!. Weekend runs with television! Let'...
13539,8723,NBA2K,Negative,@NBA2K count 7 days fucker
72724,8858,Nvidia,Negative,i just had had a huge freakout moment because ...
1816,2718,Borderlands,Positive,as Happy 4th of July!!!! Borderlands 3 Type of...


In [3]:
# Visualize split of source
training_df['SOURCE'].value_counts()

# Visualize split of sentiment
training_df['SENTIMENT'].value_counts()

Negative      22542
Positive      20832
Neutral       18318
Irrelevant    12990
Name: SENTIMENT, dtype: int64

In [ ]:
### Downsample


In [4]:
from sklearn.preprocessing import LabelEncoder

dfs = []
for df in [training_df, validation_df]:
    ### Remove Null Tweets
    df = df.dropna(subset=['TWEET'])
    
    ### Encode Source
    le = LabelEncoder()
    df['SOURCE'] = le.fit_transform(df['SOURCE'])

    ### Encode Sentiment
    df['SENTIMENT'] = le.fit_transform(df['SENTIMENT'])

    ### Drop ID
    df = df.drop(columns={'ID'})
    
    # Append dataframes
    dfs.append(df)

training_df, validation_df = dfs

C:\Users\ebrady\AppData\Local\Temp\ipykernel_19596\3254828095.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SOURCE'] = le.fit_transform(df['SOURCE'])
C:\Users\ebrady\AppData\Local\Temp\ipykernel_19596\3254828095.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SENTIMENT'] = le.fit_transform(df['SENTIMENT'])


In [6]:
### Set character threshold


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Feature Vectorization
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

train_vecs = vectorizer.fit_transform(training_df['TWEET'])
val_vecs = vectorizer.transform(validation_df['TWEET'])

In [20]:
train_vecs.shape

(73996, 18390)

In [21]:
### Combine feature vectors with data

# import pandas as pd
# import scipy.sparse as sp

# # Convert the DataFrame to a NumPy array
# train_array = training_df['SOURCE'].values
# val_array = validation_df['SOURCE'].values

# # Convert the NumPy array to a SciPy sparse CSR matrix
# train_csr_matrix = scipy.sparse.csr_matrix(train_array)
# val_csr_matrix = scipy.sparse.csr_matrix(val_array)

# # Use the `scipy.sparse.csr_matrix.vstack()` function to concatenate the two matrices
# train_preproc = sp.vstack((train_csr_matrix, train_vecs))
# val_preproc = sp.vstack((val_csr_matrix, val_vecs))

In [ ]:
from sklearn import svm
from sklearn.metrics import classification_report

x_train = train_vecs
y_train = training_df['SENTIMENT']

x_val = val_vecs
y_val_true = validation_df['SENTIMENT']

# Perform classification with SVM, kernel=linear
linear_svm = svm.SVC(kernel='linear')
linear_svm.fit(x_train, y_train)
yval_pred = linear_svm.predict(x_val)

# Results
report = classification_report(validation_df['SENTIMENT'], prediction_linear, output_dict=True)
print('positive: ', report['pos'])
print('negative: ', report['neg'])


In [ ]:
from sklearn.metrics import f1_score
f1_score(y_val_true, yval_pred)
